<h1>Importando bibliotecas</h1>

In [1]:
import pandas as pd
import numpy as np
import sys
import sklearn
from scipy.spatial import distance
import pandas as pd
from sklearn.preprocessing import Normalizer
from sklearn.metrics import log_loss
import _pickle as cPickle
from IPython.display import clear_output
from scipy.optimize import minimize, rosen, rosen_der
from timeit import default_timer as timer
from datetime import timedelta

<h1>Variáveis auxiliares</h1>

In [2]:
#variáveis inteiras
col_namesInt = ['duration', 'src_bytes', 'dst_bytes', 'land',  'logged_in',  'is_host_login', 'is_guest_login',
'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'count',
 'srv_count',
 'dst_host_count',
 'dst_host_srv_count']

#variáveis contínuas
col_namesFloat = [
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
]
#variáveis categoricas => one_hot_encoder => variáveis inteiras
categ = ['protocol_type_icmp',
 'protocol_type_tcp',
 'protocol_type_udp', 'service_IRC',
 'service_X11',
 'service_Z39_50',
 'service_aol',
 'service_auth',
 'service_bgp',
 'service_courier',
 'service_csnet_ns',
 'service_ctf',
 'service_daytime',
 'service_discard',
 'service_domain',
 'service_domain_u',
 'service_echo',
 'service_eco_i',
 'service_ecr_i',
 'service_efs',
 'service_exec',
 'service_finger',
 'service_ftp',
 'service_ftp_data',
 'service_gopher',
 'service_harvest',
 'service_hostnames',
 'service_http',
 'service_http_2784',
 'service_http_443',
 'service_http_8001',
 'service_imap4',
 'service_iso_tsap',
 'service_klogin',
 'service_kshell',
 'service_ldap',
 'service_link',
 'service_login',
 'service_mtp',
 'service_name',
 'service_netbios_dgm',
 'service_netbios_ns',
 'service_netbios_ssn',
 'service_netstat',
 'service_nnsp',
 'service_nntp',
 'service_ntp_u',
 'service_other',
 'service_pm_dump',
 'service_pop_2',
 'service_pop_3',
 'service_printer',
 'service_private',
 'service_red_i',
 'service_remote_job',
 'service_rje',
 'service_shell',
 'service_smtp',
 'service_sql_net',
 'service_ssh',
 'service_sunrpc',
 'service_supdup',
 'service_systat',
 'service_telnet',
 'service_tftp_u',
 'service_tim_i',
 'service_time',
 'service_urh_i',
 'service_urp_i',
 'service_uucp',
 'service_uucp_path',
 'service_vmnet',
 'service_whois','flag_OTH',
 'flag_REJ',
 'flag_RSTO',
 'flag_RSTOS0',
 'flag_RSTR',
 'flag_S0',
 'flag_S1',
 'flag_S2',
 'flag_S3',
 'flag_SF',
 'flag_SH']

In [3]:
tipoInteiro = {}
for s in col_namesInt:
    tipoInteiro[s] = True
for s in col_namesFloat:
    tipoInteiro[s] = False
for s in categ:
    tipoInteiro[s] = True
tipoInteiro

{'duration': True,
 'src_bytes': True,
 'dst_bytes': True,
 'land': True,
 'logged_in': True,
 'is_host_login': True,
 'is_guest_login': True,
 'wrong_fragment': True,
 'urgent': True,
 'hot': True,
 'num_failed_logins': True,
 'num_compromised': True,
 'root_shell': True,
 'su_attempted': True,
 'num_root': True,
 'num_file_creations': True,
 'num_shells': True,
 'num_access_files': True,
 'num_outbound_cmds': True,
 'count': True,
 'srv_count': True,
 'dst_host_count': True,
 'dst_host_srv_count': True,
 'serror_rate': False,
 'srv_serror_rate': False,
 'rerror_rate': False,
 'srv_rerror_rate': False,
 'same_srv_rate': False,
 'diff_srv_rate': False,
 'srv_diff_host_rate': False,
 'dst_host_same_srv_rate': False,
 'dst_host_diff_srv_rate': False,
 'dst_host_same_src_port_rate': False,
 'dst_host_srv_diff_host_rate': False,
 'dst_host_serror_rate': False,
 'dst_host_srv_serror_rate': False,
 'dst_host_rerror_rate': False,
 'dst_host_srv_rerror_rate': False,
 'protocol_type_icmp': True

In [4]:
df = pd.read_csv('datasets_adv/DoS_orig_normal.csv') #apenas auxiliar
df = df.drop('y_actual', 1)
df = df.drop('y_pred', 1)
colunas = df.columns
mapInd = {}
i = 0
for s in df.columns:
    mapInd[s] = i
    i += 1
mapInd

{'duration': 0,
 'src_bytes': 1,
 'dst_bytes': 2,
 'land': 3,
 'wrong_fragment': 4,
 'urgent': 5,
 'hot': 6,
 'num_failed_logins': 7,
 'logged_in': 8,
 'num_compromised': 9,
 'root_shell': 10,
 'su_attempted': 11,
 'num_root': 12,
 'num_file_creations': 13,
 'num_shells': 14,
 'num_access_files': 15,
 'num_outbound_cmds': 16,
 'is_host_login': 17,
 'is_guest_login': 18,
 'count': 19,
 'srv_count': 20,
 'serror_rate': 21,
 'srv_serror_rate': 22,
 'rerror_rate': 23,
 'srv_rerror_rate': 24,
 'same_srv_rate': 25,
 'diff_srv_rate': 26,
 'srv_diff_host_rate': 27,
 'dst_host_count': 28,
 'dst_host_srv_count': 29,
 'dst_host_same_srv_rate': 30,
 'dst_host_diff_srv_rate': 31,
 'dst_host_same_src_port_rate': 32,
 'dst_host_srv_diff_host_rate': 33,
 'dst_host_serror_rate': 34,
 'dst_host_srv_serror_rate': 35,
 'dst_host_rerror_rate': 36,
 'dst_host_srv_rerror_rate': 37,
 'protocol_type_icmp': 38,
 'protocol_type_tcp': 39,
 'protocol_type_udp': 40,
 'service_IRC': 41,
 'service_X11': 42,
 'service

<h1>Funções</h1>

In [5]:
def logloss(true_label, predicted, eps=1e-15):
 p = np.clip(predicted, eps, 1 - eps)
 if true_label == 1:
   return -np.log(p)
 else:
   return -np.log(1 - p)

In [6]:
def arredondarInteiros(X):
    X2 = np.array(X).reshape(1, -1)    
    for col in colunas: #dfDoS = global
        if tipoInteiro[col]:
            X2[0][mapInd[col]] = round(X2[0][mapInd[col]], 0)
    return X2

In [7]:
def obterDataset(filename, filenameOrig):
    df = pd.read_csv(filename)
    dfAux = pd.read_csv(filenameOrig)
    df = df.drop('y_actual', 1)
    df = df.drop('y_pred', 1)
    minimos = dfAux.min()
    maximos = dfAux.max()
    return (df, minimos, maximos)

In [8]:
def obterClassificador(filename):
    with open(filename, 'rb') as fid:
        model = cPickle.load(fid)
    return model

In [9]:
def obterScaler(filename):
    with open(filename, 'rb') as fid:
        scaler = cPickle.load(fid) 
    return scaler

In [10]:
def normalizar(df, scaler):
    df = df.drop('y_actual', 1)
    df = df.drop('y_pred', 1)
    X = scaler.transform(df)
    newDf = pd.DataFrame(X, columns = colunas.tolist())
    return newDf

In [11]:
def distancias(df1, df2):
    dists = []
    for i, row in df1.iterrows():
        X1 = []
        X2 = []
        for c in df1.columns:
            X1.append(df1[c][i])
            X2.append(df2[c][i])
        dists.append(distance.euclidean(X1, X2))
    return dists

In [12]:
def diferenca(df1, df2):
    dfRes = df1.copy()
    for i, row in df1.iterrows():
        for c in df1.columns:
            dfRes.at[i, c] = df2[c][i] - df1[c][i];
    return dfRes

In [13]:
model = scaler = y0 = X0 = None

In [53]:
def funcaoObjetivoInt(X):
    Xi = np.array(X0).reshape(1, -1)
    Xi = scaler.transform(Xi)
    X2 = arredondarInteiros(X)
    X2 = scaler.transform(X2)
    
    y = model.predict_proba(X2)[0][1]
    yC = model.predict(X2)
    
    if yC == y0:
        return distance.euclidean(Xi, X2)
    return 100*logloss(y0, y) + distance.euclidean(Xi, X2)

<h1>Otimização</h1>

In [54]:
def otimizar(filenameModel, filenameDf, filenameDfOrig, filenameScaler, classeReal, debugFlag=False):
    global scaler, model, X0, y0
    start = timer()
    df, minimos, maximos = obterDataset(filenameDf, filenameDfOrig)
    dfRes = pd.read_csv(filenameDf)
    scaler = obterScaler(filenameScaler)
    model = obterClassificador(filenameModel)
    y0 = classeReal
    dists = []
    
    bnds = []
    #definindo restrições
    for c in df.columns:
        bnds.append((minimos[c], maximos[c]))
    bnds = tuple(bnds)
    
    for i, row in df.iterrows():
        clear_output(wait=True)
        print(i,'/',df.shape[0],sep='', flush=True)
        X = []
        X0 = []
        for c in df.columns:
            X0.append(df[c][i])
            X.append(df[c][i])
        res = minimize(fun = funcaoObjetivoInt, x0 = X, method='Powell', options={'disp': debugFlag})
        Xaux = arredondarInteiros(res.x)
        classe = model.predict(scaler.transform(Xaux))
        j = 0
        for c in df.columns:
            dfRes.at[i, c] = Xaux[0][j]
            j += 1
        dfRes.at[i, 'y_pred'] = classe
        dists.append(distance.euclidean(X0, Xaux))
    clear_output(wait=True)
    print('Completo!', flush=True)
    end = timer()
    print(timedelta(seconds=end-start))
    return (dfRes, dists)

In [55]:
def otimizarAmostra(filenameModel, filenameDf, filenameDfOrig, filenameScaler, classeReal, i, debugFlag=False):
    global scaler, model, X0, y0
    start = timer()
    df, minimos, maximos = obterDataset(filenameDf, filenameDfOrig)
    dfRes = pd.read_csv(filenameDf)
    scaler = obterScaler(filenameScaler)
    model = obterClassificador(filenameModel)
    y0 = classeReal
    X = []
    X0 = []
    
    bnds = []
    #definindo restrições
    for c in df.columns:
        bnds.append((minimos[c], maximos[c]))
    bnds = tuple(bnds)
    
    i = 0
    #instanciando variáveis X (amostra adversária) e ponto inicial X0 (amostra original)
    for c in df.columns:
        X0.append(df[c][i])
        X.append(df[c][i])
    #Minimizando a função objetivo
    res = minimize(fun = funcaoObjetivoInt, x0 = X, method='Powell', options={'disp': debugFlag})
    Xaux = arredondarInteiros(res.x)
    classe = model.predict(scaler.transform(Xaux))
    end = timer()
    print(timedelta(seconds=end-start))
    return (Xaux, classe, distance.euclidean(X0, Xaux))

<h1>Aplicando para os datasets</h1>

<h2>Amostrais normais classificadas como DoS<h2>

In [56]:
fileClass = 'classificadores/rfc_DoS.pkl'
fileData = 'datasets_adv/DoS_orig_normal.csv'
fileDataOrig = 'DoS.csv'
fileScaler = 'scalers/scalerDoS.pkl'
filepathRes = 'datasets_resultados/Normal_DoS/'
classeReal = 0

In [57]:
#otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

In [58]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
0:44:00.299447


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-0.0,1481.0,1.0,-0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,-0.0,29.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0,0
2,1.0,1479.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,2.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0,0
4,-0.0,1480.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
5,-0.0,1480.0,1.0,-0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0,0
6,1.0,1480.0,0.0,0.0,-0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
7,1.0,1479.0,1.0,-0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
8,1.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [59]:
dfRes['y_pred'].value_counts()

0    9
Name: y_pred, dtype: int64

In [60]:
dists

[1.4142135623730951,
 1.4142135623730951,
 1.4142135623730951,
 2.0,
 1.0,
 1.0,
 1.7320508075688772,
 1.7320508075688772,
 1.0]

In [61]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
5,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
6,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
7,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
8,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [62]:
dfResNorm = normalizar(dfRes, scaler)

In [63]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [64]:
distancias(dfResNorm, dfOrigNorm)

[2.0034700369033058e-05,
 2.0034700369033058e-05,
 0.000990884059250858,
 0.0019817584139504296,
 3.74184854186156,
 1.9793260399685042e-05,
 3.7832746466401277,
 0.000991081728231687,
 0.0009908792069752148]

In [65]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-0.0,1.0,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
1,-0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0,-1
2,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
3,2.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0,-1
4,-0.0,0.0,-0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
5,-0.0,0.0,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0,-1
6,1.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
7,1.0,-1.0,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1


In [66]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

dfDif2 = diferenca(dfOrigNorm, dfResNorm)
dfDif2.to_csv(filepathRes + 'diferencas_normalizado.csv', index=False)

<h2>Amostrais DoS classificadas como normais<h2>

In [67]:
fileClass = 'classificadores/rfc_DoS.pkl'
fileData = 'datasets_adv/DoS_orig_ataques.csv'
fileDataOrig = 'DoS.csv'
fileScaler = 'scalers/scalerDoS.pkl'
filepathRes = 'datasets_resultados/DoS_Normal/'
classeReal = 1

In [68]:
otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 1, debugFlag = True)

Optimization terminated successfully.
         Current function value: 3.742027
         Iterations: 2
         Function evaluations: 11623
0:03:55.199096


(array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  5.00000000e+00,  2.00000000e+00,
          1.00000000e+00,  9.99999993e-01, -2.97633603e-10,
          3.57073924e-11,  1.00000001e+00,  7.39808424e-11,
          9.99999998e-01,  2.55000000e+02,  1.00000000e+00,
          7.03966425e-09,  2.00000000e-02,  1.05940183e-10,
          2.18269901e-11,  1.80470315e-11,  1.00000001e+00,
         -3.17077331e-10, -5.38335440e-09,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+0

In [69]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
3:04:21.139000


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,-0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,564.0,0.0,0.0,1.0,-0.0,0.0,0.0,0.0,-0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
2,-0.0,28.0,0.0,0.0,1.0,0.0,0.0,-0.0,-0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,1,1
3,0.0,28.0,-0.0,0.0,3.0,-0.0,0.0,0.0,-0.0,-0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
4,3.0,35.0,255.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,...,1.0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,1,1
5,-0.0,-0.0,0.0,0.0,2.0,0.0,-0.0,-0.0,0.0,0.0,...,1.0,-0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,1,1
6,0.0,0.0,0.0,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
7,0.0,28.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
8,0.0,28.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,1.0,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,1,1
9,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1


In [70]:
dfRes['y_pred'].value_counts()

1    28
Name: y_pred, dtype: int64

In [71]:
dists

[4.242640687119285,
 1.2248571591353523,
 2.176361654062231,
 0.5914811894689918,
 2.6863078864167758,
 2.2887007999376383,
 52.0,
 1.7054691527459072,
 2.023845578145068,
 1.9012632135038976,
 3.453726323537348,
 1.918874623896022,
 2.1945674063229883,
 2.23606797749979,
 3.000149996117052,
 1.612653235434631,
 1.4464267159139461,
 1.4144345158977634,
 1.4142135623730951,
 3.7289040162428657,
 3.16237647976421,
 3.478839613861042,
 2.1772812032201143,
 7.1414914408782355,
 2.23606797749979,
 1.0,
 3.4235215855336354,
 1.1963175255913956]

In [72]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,0.0,564.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,0.0,28.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,3.0,35.0,255.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
7,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
8,0.0,28.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
9,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [73]:
dfResNorm = normalizar(dfRes, scaler)

In [74]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [75]:
distancias(dfResNorm, dfOrigNorm)

[3.742026941264659,
 6.570418242665221,
 16.80129344231348,
 1.7250094034120838,
 4.240811521317018,
 8.737614142192077,
 0.4748359359715782,
 5.763850722133485,
 9.52891261518371,
 9.439721132314663,
 4.9422261872851285,
 5.171459103965726,
 4.315214614099119,
 0.022497952020895547,
 0.07331981240272221,
 4.127694632129671,
 5.318789682177714,
 3.7423007868209215,
 3.741859683873625,
 7.764666283650186,
 0.0643049059572519,
 4.5918637963894495,
 16.813259745731344,
 3.743272400165339,
 3.7419408071927815,
 1.9793260399685042e-05,
 7.999819267943937,
 3.2880474796328]

In [76]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,0.0,0.0,0.0,0.0,1.0,-0.0,0.0,0.0,0.0,-0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0,1
3,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,0,1
5,-0.0,-0.0,0.0,0.0,2.0,0.0,-0.0,-0.0,0.0,0.0,...,0.0,-0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0,1
6,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
7,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0,1
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [77]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

dfDif2 = diferenca(dfOrigNorm, dfResNorm)
dfDif2.to_csv(filepathRes + 'diferencas_normalizado.csv', index=False)

<h2>Amostrais normais classificadas como Probe<h2>

In [78]:
fileClass = 'classificadores/rfc_Probe.pkl'
fileData = 'datasets_adv/Probe_orig_normal.csv'
fileDataOrig = 'Probe.csv'
fileScaler = 'scalers/scalerProbe.pkl'
filepathRes = 'datasets_resultados/Normal_Probe/'
classeReal = 0

In [79]:
#otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

In [80]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
1:40:51.896964


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,-0.0,0.0,0.0,-0.0,0.0,1.0,0.0,1.0,0.0,...,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0,0
1,0.0,-0.0,0.0,0.0,-0.0,-0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,0.0,1.0,-0.0,0.0,-0.0,-0.0,2.0,-0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,-0.0,1.0,0.0,0.0,-0.0,0.0,-0.0,0.0,1.0,-0.0,...,1.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,0,0
4,0.0,1.0,3.0,0.0,0.0,-0.0,0.0,0.0,1.0,0.0,...,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0,0
5,0.0,-0.0,0.0,0.0,-0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
6,0.0,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,1.0,1.0,...,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
7,0.0,1.0,1.0,0.0,-0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
8,3.0,-0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
9,3.0,-0.0,0.0,0.0,-0.0,0.0,1.0,-0.0,0.0,0.0,...,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [81]:
dfRes['y_pred'].value_counts()

0    19
Name: y_pred, dtype: int64

In [82]:
dists

[2.3195797196963843,
 2.3715187128220645,
 2.902589389805801,
 1.5655271088907956,
 3.5121612393044326,
 2.2362692149460974,
 2.000224987775484,
 1.4142135623730951,
 4.2837629511576205,
 1.6953760850742203,
 2.0,
 0.999999999996362,
 2.703397220371146,
 3.461228106214222,
 1.7320508075688772,
 3.875338703202351,
 1.7122280818383073,
 2.449489742783178,
 3.9132147934484873]

In [83]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,-5.684342e-14,1.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,-5.684342e-14,1.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
5,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
6,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
7,-5.684342e-14,1.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
8,3.000000e+00,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
9,3.000000e+00,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [84]:
dfResNorm = normalizar(dfRes, scaler)

In [85]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [86]:
distancias(dfResNorm, dfOrigNorm)

[10.105684201092334,
 3.393207778406968,
 6.037440777997873,
 3.7494635839215147,
 6.387871495950023,
 2.1028384424324322,
 2.049849487904935,
 0.4700202653293125,
 3.9033270046200417,
 2.5038342235610154,
 2.1006238597033935,
 1.969499781474121e-07,
 4.325567112586676,
 3.94553925376651,
 2.1005488205592435,
 2.437871158794795,
 4.25568306444226,
 0.9388542492659883,
 3.884107636249003]

In [87]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,5.684342e-14,-0.0,-3.637979e-12,0.0,-0.0,0.0,1.0,0.0,1.0,0.0,...,2.168404e-19,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0,-1
1,5.684342e-14,-0.0,-3.637979e-12,0.0,-0.0,-0.0,1.0,0.0,1.0,1.0,...,2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
2,5.684342e-14,0.0,-3.637979e-12,0.0,-0.0,-0.0,2.0,-0.0,1.0,1.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
3,5.684342e-14,0.0,-3.637979e-12,0.0,-0.0,0.0,-0.0,0.0,1.0,-0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,0,-1
4,5.684342e-14,1.0,3.000000e+00,0.0,0.0,-0.0,0.0,0.0,1.0,0.0,...,2.168404e-19,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0,-1
5,5.684342e-14,-0.0,-3.637979e-12,0.0,-0.0,0.0,1.0,0.0,1.0,1.0,...,2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
6,5.684342e-14,-0.0,-3.637979e-12,0.0,-0.0,0.0,-0.0,0.0,1.0,1.0,...,2.168404e-19,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
7,5.684342e-14,0.0,0.000000e+00,0.0,-0.0,0.0,1.0,0.0,0.0,1.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
8,-5.684342e-14,-0.0,4.000000e+00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1
9,-5.684342e-14,-0.0,-3.637979e-12,0.0,-0.0,0.0,1.0,-0.0,0.0,0.0,...,2.168404e-19,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1


In [88]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

dfDif2 = diferenca(dfOrigNorm, dfResNorm)
dfDif2.to_csv(filepathRes + 'diferencas_normalizado.csv', index=False)

<h2>Amostrais Probe classificadas como normais<h2>

In [89]:
fileClass = 'classificadores/rfc_Probe.pkl'
fileData = 'datasets_adv/Probe_orig_ataques.csv'
fileDataOrig = 'Probe.csv'
fileScaler = 'scalers/scalerProbe.pkl'
filepathRes = 'datasets_resultados/Probe_Normal/'
classeReal = 1

In [90]:
otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

Optimization terminated successfully.
         Current function value: 5.086247
         Iterations: 4
         Function evaluations: 23179
0:07:14.765352


(array([[ 0.00000000e+00,  4.00000000e+01,  4.00000000e+01,
          0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  2.00000000e+00,  1.00000000e+00,
          9.79467465e-11,  1.00000005e-01,  9.95000019e-01,
          5.00000066e-01,  2.25000022e-01, -1.22692231e-10,
          5.49255284e-11,  7.00000000e+01,  1.00000000e+00,
          1.00000372e-02,  2.40000000e-01,  9.99999136e-03,
          1.57581250e-11, -4.27166506e-09,  1.72136314e-09,
          6.99999961e-02, -1.27191505e-12,  0.00000000e+00,
          0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+0

In [91]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
2:44:43.996474


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,40.0,40.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,...,1.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,-0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
2,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
3,1.0,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
4,1.0,2.0,25.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,1,1
5,-0.0,0.0,1.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
6,0.0,5.0,0.0,0.0,-0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
7,1.0,9.0,44.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
8,2.0,24.0,942.0,0.0,-0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,1,1


In [92]:
dfRes['y_pred'].value_counts()

1    31
Name: y_pred, dtype: int64

In [93]:
dists

[1.6883868254492111,
 7.14142842854285,
 1.0,
 1.599882818021531,
 4.255845979641062,
 1.414222401184538,
 1.8459076502028258,
 1.1590729300799067,
 1.422682347220288,
 1.4142135623731353,
 2.23606797749979,
 5.196152422706643,
 1.7389723978352698,
 1.4142135623731353,
 1.73205080756891,
 1.0000000000000568,
 1.0307764085555622,
 3.250003880382408,
 1.0000000000000568,
 0.8061793502952049,
 2.0889053912574185,
 1.7320508075668097,
 2.704371473049481,
 1.0,
 1.0,
 1.6896449580242316,
 3.464101615136721,
 2.3156478862173575,
 1.0463746936973843,
 2.500005009322142,
 2.093418287356243]

In [94]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-5.684342e-14,40.0,4.000000e+01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,-5.684342e-14,1.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,-5.684342e-14,1.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,1.000000e+00,6.0,2.500000e+01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
5,-5.684342e-14,1.0,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
6,-5.684342e-14,5.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
7,-5.684342e-14,9.0,4.400000e+01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
8,2.000000e+00,24.0,9.420000e+02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
9,-5.684342e-14,0.0,3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [95]:
dfResNorm = normalizar(dfRes, scaler)

In [96]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [97]:
distancias(dfResNorm, dfOrigNorm)

[5.086246743106844,
 0.08879020501069063,
 0.017755348556218242,
 5.870647745411406,
 1.1301395478090412,
 0.049105930978643623,
 4.824317169576227,
 1.999022013184211,
 0.70247071631724,
 0.00030601914344956943,
 0.4688552867333547,
 0.08877727021336135,
 0.5232533455683601,
 0.00030601914344956943,
 0.0003060192068267971,
 0.0003060191137047863,
 0.9648140757068481,
 6.883078990134348,
 0.0003060191137047863,
 3.236456847271974,
 3.9257373802071323,
 0.0003060192068267971,
 6.313092482611981,
 0.012428118303291215,
 0.017755348556218242,
 5.8771758627015895,
 0.03728561074731588,
 4.7256916309508,
 2.3732872119318698,
 4.588932083280712,
 10.354769036824619]

In [98]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,5.684342e-14,0.0,-3.637979e-12,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,5.684342e-14,-1.0,-3.637979e-12,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,5.684342e-14,-0.0,-3.637979e-12,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,...,2.168404e-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,1.000000e+00,-1.0,-3.637979e-12,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,0.000000e+00,-4.0,0.000000e+00,0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0,1
5,5.684342e-14,-1.0,0.000000e+00,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
6,5.684342e-14,0.0,-3.637979e-12,0.0,-0.0,0.0,1.0,0.0,0.0,1.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
7,1.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,0.000000e+00,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
8,0.000000e+00,0.0,3.637979e-12,0.0,-0.0,0.0,1.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
9,1.000000e+00,1.0,-3.637979e-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.168404e-19,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,1


In [99]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

dfDif2 = diferenca(dfOrigNorm, dfResNorm)
dfDif2.to_csv(filepathRes + 'diferencas_normalizado.csv', index=False)

<h2>Amostrais normais classificadas como R2L<h2>

In [100]:
fileClass = 'classificadores/rfc_R2L.pkl'
fileData = 'datasets_adv/R2L_orig_normal.csv'
fileDataOrig = 'R2L.csv'
fileScaler = 'scalers/scalerR2L.pkl'
filepathRes = 'datasets_resultados/Normal_R2L/'
classeReal = 0

In [101]:
#otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

In [102]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
7:18:13.926427


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,105.0,146.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,-0.0,687.0,-0.0,0.0,0.0,0.0,-0.0,0.0,1.0,-0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,-0.0,105.0,145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,1.0,105.0,145.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,-0.0,105.0,145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,-0.0,105.0,145.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
86,60.0,129.0,179.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0,0
87,28.0,6291668.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
88,-0.0,105.0,145.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [103]:
dfRes['y_pred'].value_counts()

0    90
Name: y_pred, dtype: int64

In [104]:
dists

[2.0,
 1.0,
 2.0,
 1.7320508075688772,
 1.0,
 2.449489742783178,
 5.0,
 1.0,
 3.1622776601683795,
 5.0,
 4.242640687119285,
 3.1622776601683795,
 2.0,
 2.23606797749979,
 3.4641016151377544,
 2.0,
 1.0,
 2.23606797749979,
 2.23606797749979,
 3.1622776601683795,
 2.23606797749979,
 3.3166247903554,
 4.242640687119285,
 1.0000000000000002,
 1.0,
 1.0,
 1.0,
 4.123105625617661,
 2.449489742783178,
 1.4142135623730951,
 1.0,
 1.0,
 2.0,
 1.0,
 6.782329983125268,
 5.196152422706632,
 2.449489742783178,
 1.7320508075688772,
 2.23606797749979,
 1.4142135623730951,
 4.47213595499958,
 1.7320508075688772,
 1.4142135623730951,
 1.7320508075688772,
 1.0,
 3.3166247903554,
 1.7320508075688772,
 4.123105625617661,
 1.7320508075688772,
 1.0,
 1.0,
 1.0,
 2.6457513110645907,
 2.0,
 3.3166247903554,
 1.0,
 1.4142135623730951,
 1.7320508075688772,
 1.0,
 5.385164807134504,
 1.0,
 3.605551275463989,
 2.6457513110645907,
 2.449489742783178,
 3.7416573867739413,
 3.3166247903554,
 1.0,
 1.4142135623730951

In [105]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
1,0.0,687.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
2,0.0,105.0,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
3,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
4,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
86,60.0,129.0,179.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
87,28.0,6291668.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1
88,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,0,1


In [106]:
dfResNorm = normalizar(dfRes, scaler)

In [107]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [108]:
distancias(dfResNorm, dfOrigNorm)

[2.238830045366628,
 2.2120049462268536,
 1.992057508691525e-05,
 0.3446280424206394,
 9.960287543457624e-06,
 0.344630526033051,
 0.003777063690009269,
 9.960287543457624e-06,
 2.2392495374843646,
 0.003777063690009269,
 2.606386876929899,
 2.239000082035465,
 0.001510825476003702,
 2.316902576638117,
 1.0338819190934574,
 0.037250073165760914,
 9.960287543457624e-06,
 0.34462721493216303,
 0.34462721493216303,
 2.317143645503555,
 0.689254428784819,
 2.4416955449250635,
 1.0345535764373495,
 7.657560628597408,
 9.960287543457624e-06,
 9.960287543457624e-06,
 9.960287543457624e-06,
 16.906898315425725,
 2.2386901972814295,
 1.0214493510991902e-05,
 9.960287543457624e-06,
 9.960287543457624e-06,
 1.992057508691525e-05,
 2.2646301547357695e-06,
 1.723138721244805,
 1.7231362373678647,
 2.3169026997870996,
 2.2387676724025765,
 0.344627214932163,
 2.21206807381092,
 2.441980136007765,
 0.0007554817937006558,
 0.0007554783994684577,
 2.238690324644197,
 2.2646301547357695e-06,
 2.31697792

In [109]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
1,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,1.0,-0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
2,-0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
3,1.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
4,-0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.0,2.168404e-19,0.0,0.0,0.0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,-0.0,0.0,-1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
86,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,-0.0,2.168404e-19,0.0,0.0,0.0,0,-1
87,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1
88,-0.0,0.0,-1.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,-1


In [110]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

dfDif2 = diferenca(dfOrigNorm, dfResNorm)
dfDif2.to_csv(filepathRes + 'diferencas_normalizado.csv', index=False)

<h2>Amostrais R2L classificadas como normais<h2>

In [111]:
fileClass = 'classificadores/rfc_R2L.pkl'
fileData = 'datasets_adv/R2L_orig_ataques.csv'
fileDataOrig = 'R2L.csv'
fileScaler = 'scalers/scalerR2L.pkl'
filepathRes = 'datasets_resultados/R2L_Normal/'
classeReal = 1

In [112]:
otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

Optimization terminated successfully.
         Current function value: 0.000755
         Iterations: 2
         Function evaluations: 11749
0:03:16.330333


(array([[ 3.00000000e+00,  1.23000000e+02,  1.78000000e+02,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  2.20000000e+01,  6.00000000e+00,
          2.70000000e-01,  1.40000000e-01,  5.00000000e-02,
          3.46944695e-18, -1.73472348e-18,  0.00000000e+00,
          9.00000000e-02,  3.30000000e-01,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+0

In [113]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
11:30:53.793651


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,3.0,123.0,178.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,-0.0,10.0,35.0,0.0,0.0,0.0,-0.0,-0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
2,10.0,118.0,2425.0,0.0,-0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,1,1
3,218.0,181832.0,24172.0,0.0,-0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
4,1.0,146.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.0,105.0,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
145,0.0,12.0,0.0,0.0,-0.0,0.0,0.0,1.0,-0.0,-0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
146,0.0,176412.0,0.0,0.0,-0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
147,3.0,105.0,147.0,0.0,-0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1


In [114]:
dfRes['y_pred'].value_counts()

1    149
Name: y_pred, dtype: int64

In [115]:
dists

[1.0000000000000284,
 1.4142135623730951,
 1.4142135623730958,
 1.0000000000000007,
 1.0,
 1.0000000000000002,
 1.0000000000000004,
 2.000000000000001,
 1.4142135623730951,
 1.899638163400116,
 3.1622776601683795,
 1.000000000000001,
 2.23606797749979,
 1.4142135623730951,
 1.0000000000000007,
 1.000000000000001,
 2.831316118636831,
 1.0,
 1.000000000000001,
 1.0000000000000122,
 3.0000000000000013,
 1.000000000000001,
 1.0000000000000007,
 1.4142135623730954,
 1.0000000000000009,
 1.0,
 1.4142135623730954,
 1.000000000000001,
 4.242640687119285,
 1.0,
 1.0,
 2.0,
 1.7333350575318838,
 1.4142135623730983,
 2.6457513110645907,
 3.0594117228048914,
 1.0000000000000002,
 3.0,
 1.4142135623730954,
 3.605551275463989,
 1.4142135623730956,
 1.0000000000000007,
 1.0000000000000007,
 1.4142135623730954,
 2.2360679774997902,
 1.0000000000000007,
 1.0000000000000002,
 1.0000000000000004,
 1.4142135623730954,
 1.4142135623730963,
 3.1622776601683795,
 1.0000000000000007,
 2.0,
 1.000000000000001,

In [116]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,4.0,123.0,178.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
1,0.0,10.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
2,10.0,118.0,2425.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
3,218.0,181832.0,24172.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
4,0.0,146.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
145,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
146,0.0,176412.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0
147,0.0,105.0,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-2.168404e-19,0.0,0.0,0.0,1,0


In [117]:
dfResNorm = normalizar(dfRes, scaler)

In [118]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [119]:
distancias(dfResNorm, dfOrigNorm)

[0.0007554127380018649,
 7.970646188533098,
 16.995172545763587,
 52.28566177022754,
 0.0007554127380018649,
 16.85060604186521,
 16.85060604186521,
 19.943959780677275,
 18.50894807272389,
 18.691197463149887,
 16.85060619425841,
 16.85060604186521,
 2.316902576638117,
 18.50894807272389,
 52.28566177022754,
 16.85060604186521,
 11.381606861618218,
 9.960287543457624e-06,
 16.85060604186521,
 52.28566177022754,
 52.285661770231535,
 16.85060604186521,
 52.28566177022754,
 16.85060604186815,
 16.85060604186521,
 9.960287543457624e-06,
 16.995172545763587,
 16.85060604186521,
 52.2856618575409,
 9.960287543457624e-06,
 9.960287543457624e-06,
 0.001510825476003702,
 19.952132342951195,
 18.508948072723893,
 16.998666420038514,
 18.689543465154095,
 16.850606041865205,
 18.640702563491597,
 18.50894807272389,
 7.665377678595607,
 18.50894807272389,
 16.85060604186521,
 16.85060604186521,
 18.50894807272389,
 52.28566177022773,
 16.85060604186521,
 7.657560628597407,
 16.85060604186521,
 1

In [120]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
1,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
2,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,1.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,-0.0,0.0,0,1
3,0.0,0.0,0.0,0.0,-0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
145,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,1.0,-0.0,-0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
146,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1
147,3.0,0.0,0.0,0.0,-0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.168404e-19,0.0,0.0,0.0,0,1


In [121]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

dfDif2 = diferenca(dfOrigNorm, dfResNorm)
dfDif2.to_csv(filepathRes + 'diferencas_normalizado.csv', index=False)

<h2>Amostrais normais classificadas como U2R<h2>

In [122]:
fileClass = 'classificadores/rfc_U2R.pkl'
fileData = 'datasets_adv/U2R_orig_normal.csv'
fileDataOrig = 'U2R.csv'
fileScaler = 'scalers/scalerU2R.pkl'
filepathRes = 'datasets_resultados/Normal_U2R/'
classeReal = 0

In [123]:
#otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

In [124]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
0:00:01.099494


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred


In [125]:
dfRes['y_pred'].value_counts()

Series([], Name: y_pred, dtype: int64)

In [126]:
dists

[]

In [127]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred


In [128]:
dfResNorm = normalizar(dfRes, scaler)

ValueError: Found array with 0 sample(s) (shape=(0, 122)) while a minimum of 1 is required by StandardScaler.

In [ ]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [ ]:
distancias(dfResNorm, dfOrigNorm)

In [ ]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

In [ ]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

dfDif2 = diferenca(dfOrigNorm, dfResNorm)
dfDif2.to_csv(filepathRes + 'diferencas_normalizado.csv', index=False)

<h2>Amostrais U2R classificadas como normais<h2>

In [129]:
fileClass = 'classificadores/rfc_U2R.pkl'
fileData = 'datasets_adv/U2R_orig_ataques.csv'
fileDataOrig = 'U2R.csv'
fileScaler = 'scalers/scalerU2R.pkl'
filepathRes = 'datasets_resultados/U2R_Normal/'
classeReal = 1

In [130]:
otimizarAmostra(fileClass, fileData, fileDataOrig, fileScaler, classeReal, 0, debugFlag = True)

Optimization terminated successfully.
         Current function value: 40.323820
         Iterations: 3
         Function evaluations: 17577
0:06:25.342492


(array([[ 1.63000000e+02, -1.62000000e+05, -2.55698000e+05,
         -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
          2.00000000e+00, -0.00000000e+00,  1.00000000e+00,
          1.00000000e+00,  1.00000000e+00, -0.00000000e+00,
         -0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -1.00000000e+00, -0.00000000e+00,
          2.73857263e-03,  9.97105400e-04,  3.76612565e-02,
          3.70342348e-02,  9.78772073e-01,  1.84798552e-02,
          6.69463069e-03,  1.93000000e+02, -4.20000000e+01,
         -1.90731851e-01,  9.31739254e-02,  4.91684586e-02,
         -1.81579049e-03,  2.98464929e-03,  1.30135696e-03,
          3.27467004e-02,  3.44931549e-02,  0.00000000e+00,
          1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+0

In [131]:
dfRes, dists = otimizar(fileClass, fileData, fileDataOrig, fileScaler, classeReal, debugFlag = True)
dfRes

Completo!
2:15:42.476570


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,163.0,-162000.0,-255698.0,-0.0,0.0,-0.0,2.0,-0.0,1.0,1.0,...,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
1,5601.0,-101707.0,2603796.0,-0.0,0.0,-0.0,2.0,1.0,1.0,14.0,...,1.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
2,85.0,-162035.0,-255155.0,-0.0,0.0,-0.0,2.0,1.0,1.0,-0.0,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
3,163.0,-106885.0,2523255.0,-0.0,0.0,2.0,3.0,-0.0,1.0,23.0,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
4,28.0,-162567.0,-305589.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
5,75.0,-162372.0,-255232.0,0.0,0.0,-0.0,1.0,-0.0,1.0,-1.0,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
6,93.0,-161858.0,-184468.0,-0.0,0.0,-0.0,6.0,-0.0,1.0,1.0,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
7,105.0,-161273.0,-263612.0,-0.0,0.0,-0.0,3.0,-0.0,1.0,-1.0,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
8,28.0,-125452.0,-305589.0,-0.0,0.0,-0.0,1.0,-0.0,-0.0,-1.0,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1
9,210.0,-98285.0,1667029.0,-0.0,0.0,2.0,6.0,-0.0,1.0,1.0,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,1,1


In [132]:
dfRes['y_pred'].value_counts()

1    25
Name: y_pred, dtype: int64

In [133]:
dists

[2.2653423625551823,
 3.0465935138625153,
 2.014948197757796,
 2.8599364796344164,
 4.416901219056955,
 1.9252881033952078,
 2.117299559441989,
 2.0055389533130903,
 2.4355774095235496,
 3.33574220916109,
 2.2124691431152055,
 3.1059962205828358,
 2.6846581582510294,
 2.3461600441032147,
 2.010830454189108,
 2.8419682686567507,
 3.37665971980022,
 6.997300216617637,
 2.9134923957030234,
 1.966232080383229,
 2.4557376553368107,
 2.411093131293628,
 2.1996712767306232,
 2.4957693278867015,
 2.1790492536312014]

In [134]:
dfOrig = pd.read_csv(fileData)
scaler = obterScaler(fileScaler)
dfOrig

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,162.856596,-1.619996e+05,-2.556977e+05,-0.000007,0.0,-0.000021,1.831071,-0.000117,0.918422,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
1,5601.013191,-1.017074e+05,2.603796e+06,-0.000007,0.0,-0.000021,-0.017121,0.964415,0.918422,13.818322,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
2,85.168645,-1.620350e+05,-2.551546e+05,-0.000007,0.0,-0.000021,1.831071,0.964415,0.918422,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
3,162.856596,-1.068854e+05,2.523255e+06,-0.000007,0.0,1.802761,0.906975,-0.000117,0.918422,23.054882,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
4,27.529197,-1.625670e+05,-3.055893e+05,-0.000007,0.0,-0.000021,-0.017121,-0.000117,-0.158884,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
5,75.144393,-1.623719e+05,-2.552321e+05,-0.000007,0.0,-0.000021,0.906975,-0.000117,0.918422,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
6,92.686834,-1.618577e+05,-1.844684e+05,-0.000007,0.0,-0.000021,6.451552,-0.000117,0.918422,0.887138,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
7,105.217149,-1.612725e+05,-2.636121e+05,-0.000007,0.0,-0.000021,2.755168,-0.000117,0.918422,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
8,27.529197,-1.254518e+05,-3.055893e+05,-0.000007,0.0,-0.000021,-0.017121,-0.000117,-0.158884,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0
9,210.471792,-9.828489e+04,1.667029e+06,-0.000007,0.0,1.802761,5.527456,-0.000117,0.918422,-0.036518,...,0.037389,0.021567,0.000615,-0.000374,-0.000113,-0.000041,-0.433582,0.003064,1,0


In [135]:
dfResNorm = normalizar(dfRes, scaler)

In [136]:
dfOrigNorm = normalizar(dfOrig, scaler)

In [137]:
distancias(dfResNorm, dfOrigNorm)

[40.32382004730727,
 12.224284036676387,
 28.550076152474716,
 12.716865431574732,
 40.82686698789178,
 28.540996967738106,
 12.806694923370346,
 28.541165653446033,
 40.32925935431805,
 12.657720062391567,
 40.32381992283466,
 20.699631869691164,
 5.764349176892495,
 40.32614936174798,
 44.93305078093785,
 40.3222383718222,
 35.83887910162842,
 21.02307545245903,
 15.855495329668464,
 40.32376122578004,
 40.32925949500254,
 20.984370388628246,
 40.30258347784598,
 20.54416249098917,
 40.32614892607053]

In [138]:
dfDif = diferenca(dfOrig, dfRes)
dfDif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.143404,-0.445630,-0.299389,0.000007,0.0,0.000021,0.168929,0.000117,0.081578,1.036518,...,-0.037389,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
1,-0.013191,0.357652,0.138515,0.000007,0.0,0.000021,2.017121,0.035585,0.081578,0.181678,...,0.962611,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
2,-0.168645,0.020368,-0.442550,0.000007,0.0,0.000021,0.168929,0.035585,0.081578,0.036518,...,-0.037389,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
3,0.143404,0.393370,-0.489807,0.000007,0.0,0.197239,2.093025,0.000117,0.081578,-0.054882,...,-0.037389,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
4,0.470803,0.010339,0.279599,0.000007,0.0,0.000021,0.017121,0.000117,0.158884,0.036518,...,-0.037389,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
5,-0.144393,-0.052651,0.149330,0.000007,0.0,0.000021,0.093025,0.000117,0.081578,-0.963482,...,-0.037389,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
6,0.313166,-0.309623,0.354561,0.000007,0.0,0.000021,-0.451552,0.000117,0.081578,0.112862,...,-0.037389,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
7,-0.217149,-0.498591,0.072395,0.000007,0.0,0.000021,0.244832,0.000117,0.081578,-0.963482,...,-0.037389,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
8,0.470803,-0.156641,0.279599,0.000007,0.0,0.000021,1.017121,0.000117,0.158884,-0.963482,...,-0.037389,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1
9,-0.471792,-0.111162,-0.091475,0.000007,0.0,0.197239,0.472544,0.000117,0.081578,1.036518,...,-0.037389,-0.021567,-0.000615,0.000374,0.000113,0.000041,0.433582,-0.003064,0,1


In [139]:
#exportando datasets
dfOrig.to_csv(filepathRes + 'orig.csv', index=False)
dfOrigNorm.to_csv(filepathRes + 'orig_normalizado.csv', index=False)
dfRes.to_csv(filepathRes + 'advers.csv', index=False)
dfResNorm.to_csv(filepathRes + 'advers_normalizado.csv', index=False)
dfDif.to_csv(filepathRes + 'diferencas.csv', index=False)

dfDif2 = diferenca(dfOrigNorm, dfResNorm)
dfDif2.to_csv(filepathRes + 'diferencas_normalizado.csv', index=False)